# Part 2: Training

In this part we will show how using MLRun's **Feature Store** we can easily define a **Feature Vector** and create the dataset we need to run our training process.  
By the end of this tutorial you’ll learn how to:
- Combine multiple data sources to a single Feature Vector
- Create training dataset
- Create a model using an MLRun Hub function

In [1]:
import mlrun

project_name_base = 'fraud-demo'

# Initialize the MLRun project object
project = mlrun.get_or_create_project(project_name_base, context="./", user_project=True)

> 2021-09-10 01:31:05,499 [info] loaded project fraud-demo from MLRun DB


## Create Feature Vector  
In this section we will create our Feature Vector.  
The Feature vector will have a `name` so we can reference to it later via the URI or our serving function, and a list of `features` from the available FeatureSets.  We can add a feature from a feature set by adding `<FeatureSet>.<Feature>` to the list, or add `<FeatureSet>.*` to add all the FeatureSet's available features.  

By default, the first FeatureSet in the feature list will act as the spine. meaning that all the other features will be joined to it.  
For example, in this instance we use the early_sense sensor data as our spine, so for each early_sense event we will create produce a row in the resulted Feature Vector.

In [2]:
# Define the list of features we will be using
features = ['transactions.amount_max_2h', 
            'transactions.amount_sum_2h', 
            'transactions.amount_count_2h',
            'transactions.amount_avg_2h', 
            'transactions.amount_max_12h', 
            'transactions.amount_sum_12h',
            'transactions.amount_count_12h', 
            'transactions.amount_avg_12h', 
            'transactions.amount_max_24h',
            'transactions.amount_sum_24h', 
            'transactions.amount_count_24h', 
            'transactions.amount_avg_24h',
            'transactions.es_transportation_count_14d', 
            'transactions.es_health_count_14d',
            'transactions.es_otherservices_count_14d', 
            'transactions.es_food_count_14d',
            'transactions.es_hotelservices_count_14d', 
            'transactions.es_barsandrestaurants_count_14d',
            'transactions.es_tech_count_14d', 
            'transactions.es_sportsandtoys_count_14d',
            'transactions.es_wellnessandbeauty_count_14d', 
            'transactions.es_hyper_count_14d',
            'transactions.es_fashion_count_14d', 
            'transactions.es_home_count_14d', 
            'transactions.es_travel_count_14d', 
            'transactions.es_leisure_count_14d',
#             'transactions.age_mapped',
            'transactions.gender_F',
            'transactions.gender_M',
            'transactions.step', 
            'transactions.amount', 
            'transactions.timestamp_hour',
            'transactions.timestamp_day_of_week',
            'events.*']

In [3]:
# Import MLRun's Feature Store
import mlrun.feature_store as fstore

# Define the feature vector name for future reference
feature_vector_name = 'transactions-fraud'

# Define the feature vector using our Feature Store (fstore)
fv = fstore.FeatureVector(feature_vector_name, 
                          features, 
                          label_feature="labels.label",
                          description='Predicting a fraudulent transaction')

# Save the feature vector in the Feature Store
fv.save()

## Produce training dataset 

In [4]:
# Import the Parquet Target so we can directly save our dataset as a file
from mlrun.datastore.targets import ParquetTarget

# Get offline feature vector
# will return a pandas dataframe and save the dataset to parquet so a 
# training job can train on it
dataset = fstore.get_offline_features(feature_vector_name, target=ParquetTarget())

> 2021-09-10 01:31:06,749 [info] wrote target: {'name': 'parquet', 'kind': 'parquet', 'path': 'v3io:///projects/fraud-demo-admin/FeatureStore/transactions-fraud/parquet/vectors/transactions-fraud-latest.parquet', 'status': 'ready', 'updated': '2021-09-10T01:31:06.749852+00:00', 'size': 2027960}


In [5]:
# Preview our dataset
df = dataset.to_dataframe()

In [6]:
df.tail(5)

,amount_max_2h,amount_sum_2h,amount_count_2h,amount_avg_2h,amount_max_12h,amount_sum_12h,amount_count_12h,amount_avg_12h,amount_max_24h,amount_sum_24h,...,gender_F,gender_M,step,amount,timestamp_hour,timestamp_day_of_week,event_details_change,event_login,event_password_change,label
49995,30.69,30.69,1.0,30.69,54.27,84.96,2.0,42.480000,54.27,183.48,...,1,0,56,30.69,1,4,1.0,0.0,0.0,0
49996,14.33,14.33,1.0,14.33,14.33,22.84,2.0,11.420000,14.33,44.03,...,1,0,68,14.33,1,4,1.0,0.0,0.0,0
49997,38.10,38.10,1.0,38.10,62.75,187.49,4.0,46.872500,62.75,294.69,...,0,1,65,38.10,1,4,0.0,0.0,1.0,0
49998,60.41,60.41,1.0,60.41,60.41,60.41,1.0,60.410000,60.41,60.41,...,1,0,122,60.41,1,4,0.0,0.0,1.0,0
49999,41.33,41.33,1.0,41.33,45.70,114.40,3.0,38.133333,45.70,196.28,...,0,1,0,41.33,1,4,0.0,1.0,0.0,0


## Run training with AutoML as a cluster job

In [7]:
from mlrun.platforms import auto_mount

# Import the SKLearn based training function from our functions hub
fn = mlrun.import_function('hub://sklearn-classifier').apply(auto_mount())

In [8]:
# Prepare the parameters list for the training function
# We use 3 different models to test on our dataset
model_list = {"model_name": ['transaction_fraud_rf', 
                             'transaction_fraud_xgboost', 
                             'transaction_fraud_adaboost'],
              
              "model_pkg_class": ['sklearn.ensemble.RandomForestClassifier',
                                  'sklearn.ensemble.GradientBoostingClassifier',
                                  'sklearn.ensemble.AdaBoostClassifier']}

# Define the training task, including our feature vector, label and hyperparams definitions
task = mlrun.new_task('training', 
                      inputs={'dataset': fv.uri},
                      params={'label_column': 'label'}
                     )

task.with_hyper_params(model_list, strategy='list', selector='max.accuracy')

# Run the function 
fn.spec.image = 'mlrun/mlrun'
train = fn.run(task, local=False)

> 2021-09-10 01:31:07,097 [info] starting run training uid=dd78759f1fb24178b4b89bc7090cee69 DB=http://mlrun-api:8080
> 2021-09-10 01:31:07,342 [info] Job is running in the background, pod: training-h2g7b
> 2021-09-10 01:31:29,431 [info] best iteration=1, used criteria max.accuracy
> 2021-09-10 01:31:30,295 [info] run executed, status=completed
final state: completed


project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
fraud-demo-admin,...090cee69,0,Sep 10 01:31:13,completed,training,v3io_user=adminkind=jobowner=admin,dataset,label_column=label,best_iteration=1accuracy=0.995026778882938test-error=0.004973221117061974rocauc=0.9138470451911935brier_score=0.005322742922723794f1-score=0.6976744186046512precision_score=0.8333333333333334recall_score=0.6,test_setprobability-calibrationconfusion-matrixfeature-importancesprecision-recall-binaryroc-binarymodeliteration_results


> 2021-09-10 01:31:36,985 [info] run executed, status=completed


## Perform feature selection process on a sample of the dataset (Using mlrun marketplace function)

As part of our data science process we will try and reduce the training dataset's size to get rid of bad or unuseful features and save computation time.

We will use our ready-made feature selection function from our hub [`hub://feature_selection`](https://github.com/mlrun/functions/blob/development/feature_selection/feature_selection.ipynb) to select the best features to keep on a sample from our dataset and run the function on that.


In [9]:
feature_selection_fn = mlrun.import_function('hub://feature_selection')
feature_selection_fn.apply(auto_mount())

feature_selection_run = feature_selection_fn.run(
            params={'k': 5, 'sample_ratio':0.25,
                    'output_vector_name':feature_vector_name + "-short",
                   'ignore_type_errors': True},
            inputs={'df_artifact': fv.uri},
            name='feature_extraction',
            handler='feature_selection',
    local=False)

> 2021-09-10 01:31:37,187 [info] starting run feature_extraction uid=7239c758fbc74d31b463cc045703d15f DB=http://mlrun-api:8080
> 2021-09-10 01:31:37,444 [info] Job is running in the background, pod: feature-extraction-ctq2x
> 2021-09-10 01:31:47,910 [info] Couldn't calculate chi2 because of: Input X must be non-negative.
> 2021-09-10 01:31:52,323 [info] votes needed to be selected: 3
> 2021-09-10 01:31:53,104 [info] wrote target: {'name': 'parquet', 'kind': 'parquet', 'path': 'v3io:///projects/fraud-demo-admin/FeatureStore/transactions-fraud-short/parquet/vectors/transactions-fraud-short-latest.parquet', 'status': 'ready', 'updated': '2021-09-10T01:31:53.104703+00:00', 'size': 671011}
> 2021-09-10 01:31:53,294 [info] run executed, status=completed
Pass k=5 as keyword args. From version 0.25 passing these as positional arguments will result in an error
Liblinear failed to converge, increase the number of iterations.
final state: completed


project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
fraud-demo-admin,...5703d15f,0,Sep 10 01:31:44,completed,feature_extraction,v3io_user=adminkind=jobowner=adminhost=feature-extraction-ctq2x,df_artifact,k=5sample_ratio=0.25output_vector_name=transactions-fraud-shortignore_type_errors=True,top_features_vector=store://feature-vectors/fraud-demo-admin/transactions-fraud-short,f_classifmutual_info_classiff_regressionLinearSVCLogisticRegressionExtraTreesClassifierfeature_scoresmax_scaled_scores_feature_scoresselected_features_countselected_features


> 2021-09-10 01:31:56,906 [info] run executed, status=completed


In [10]:
top_features_df = mlrun.get_dataitem(feature_selection_run.outputs['top_features_vector']).as_df()
top_features_df.tail()

,amount_max_2h,amount_sum_2h,amount_count_2h,amount_avg_2h,amount_max_12h,label
49995,30.69,30.69,1.0,30.69,54.27,0
49996,14.33,14.33,1.0,14.33,14.33,0
49997,38.10,38.10,1.0,38.10,62.75,0
49998,60.41,60.41,1.0,60.41,60.41,0
49999,41.33,41.33,1.0,41.33,45.70,0


## Train ensemble of models with top features

In [11]:
# Defining our training task, including our feature vector, label and hyperparams definitions
task = mlrun.new_task('training', 
                      inputs={'dataset': feature_selection_run.outputs['top_features_vector']},
                      params={'label_column': 'label'}
                     )
task.with_hyper_params(model_list, strategy='list', selector='max.accuracy')

run = fn.run(task)

> 2021-09-10 01:31:57,008 [info] starting run training uid=c98b47380dcf4d7bac40e1b03c69d1ee DB=http://mlrun-api:8080
> 2021-09-10 01:31:57,245 [info] Job is running in the background, pod: training-m989j
> 2021-09-10 01:32:12,883 [info] best iteration=3, used criteria max.accuracy
> 2021-09-10 01:32:13,812 [info] run executed, status=completed
final state: completed


project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
fraud-demo-admin,...3c69d1ee,0,Sep 10 01:32:03,completed,training,v3io_user=adminkind=jobowner=admin,dataset,label_column=label,best_iteration=3accuracy=0.9909262759924385test-error=0.009073724007561436rocauc=0.9363335533704251brier_score=0.20638134186518234f1-score=0.5636363636363636precision_score=0.7209302325581395recall_score=0.4626865671641791,test_setprobability-calibrationconfusion-matrixfeature-importancesprecision-recall-binaryroc-binarymodeliteration_results


> 2021-09-10 01:32:16,615 [info] run executed, status=completed


## Done!

You've completed Part 2 of the model traininig with the feature store.
Proceed to [Part 3](3-model-serving.ipynb) to learn how to deploy and monitor the model.